In [1]:
from langchain_groq import ChatGroq

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("Who is Virat kohli?")
print(response.content)

Virat Kohli is a renowned Indian international cricketer who has been one of the most successful batsmen in the world. Born on November 5, 1988, in Delhi, India, Kohli is widely regarded as one of the greatest batsmen of all time.

Kohli made his international debut in 2008 and has since become a key player for the Indian national team. He has played in all formats of the game, including Test matches, One-Day Internationals (ODIs), and Twenty20 Internationals (T20Is).

Some of Kohli's notable achievements include:

1. **Highest run-scorer in ODIs**: Kohli is the fastest batsman to reach 12,000 runs in ODIs and has the highest number of centuries in the format.
2. **Test captaincy**: Kohli has been the captain of the Indian Test team and has led the team to several victories, including a historic series win in Australia in 2018-19.
3. **IPL success**: Kohli has been a successful captain of the Royal Challengers Bangalore (RCB) team in the Indian Premier League (IPL) and has led the team

In [10]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-20370?")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Auszubildende/r 2024 im Einzelhandel - 37.5H - Metzingen (w/m/d)

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsInternshipsAuszubildende/r 2024 im Einzelhandel - 37.5H - Metz

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [12]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Auszubildende/r 2024 im Einzelhandel',
 'experience': 'No specific experience required',
 'skills': ['Passion for NIKE and/or love for sports',
  'Competence for excellent customer service',
  'Good German and English skills',
  'Good communication and teamwork skills',
  'Proactivity, openness, motivation, and passion for putting customers first'],
 'description': "As an Auszubildende/r, you will be responsible for creating a unique brand experience for our customers through excellent customer service and product knowledge. You will support various areas of the store, recommend products to customers, and use your knowledge of digital devices to connect online and offline sales and services. You will also learn about Nike's supply chain and retail processes, develop quality assurance measures, and be a brand ambassador for NIKE initiatives."}

In [13]:
type(json_res)

dict

In [14]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [15]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [16]:
job = json_res
job['skills']

['Passion for NIKE and/or love for sports',
 'Competence for excellent customer service',
 'Good German and English skills',
 'Good communication and teamwork skills',
 'Proactivity, openness, motivation, and passion for putting customers first']

In [17]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [18]:
job

{'role': 'Auszubildende/r 2024 im Einzelhandel',
 'experience': 'No specific experience required',
 'skills': ['Passion for NIKE and/or love for sports',
  'Competence for excellent customer service',
  'Good German and English skills',
  'Good communication and teamwork skills',
  'Proactivity, openness, motivation, and passion for putting customers first'],
 'description': "As an Auszubildende/r, you will be responsible for creating a unique brand experience for our customers through excellent customer service and product knowledge. You will support various areas of the store, recommend products to customers, and use your knowledge of digital devices to connect online and offline sales and services. You will also learn about Nike's supply chain and retail processes, develop quality assurance measures, and be a brand ambassador for NIKE initiatives."}

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Your Retail Experience with AtliQ's Automation Solutions

Dear Hiring Manager,

I came across the job description for Auszubildende/r 2024 im Einzelhandel at NIKE, and I was impressed by the emphasis on creating a unique brand experience through excellent customer service and product knowledge. As a Business Development Executive at AtliQ, I believe our company can help NIKE achieve this goal by streamlining retail processes and enhancing the overall customer experience.

At AtliQ, we specialize in developing tailored automation solutions that facilitate the seamless integration of business processes. Our expertise in Java, Magento, and Python can help optimize supply chain management, improve quality assurance measures, and enhance the brand's online and offline presence.

Our portfolio showcases our capabilities in developing scalable and efficient solutions for various industries, including retail. Some of our notable projects include:

* Magento-based e-commerce so